In [8]:
import numpy as np
import scipy.io.wavfile as wav
from matplotlib import pyplot as plt
import IPython.display as ipd
import IPython
import librosa
import pyaudio
import wave
import speech_recognition as sr
import random
import click

In [9]:
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "output.wav"

def recording_now():
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)

    print("* recording")

    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("* done recording")
    stream.stop_stream()
    stream.close()
    p.terminate()
    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()

In [10]:
def random_int(min, max):
    x = random.random()
    y = 1.0 / (max - min)
    z = int(x // y)
    return min + z

def random_permutation(array):
    length = len(array)
    for i in range(length):
        b = random_int(i, length)
        array[i], array[b] = array[b], array[i]

In [11]:
def detect():
    r = sr.Recognizer()
    output = sr.AudioFile('output.wav')
    with output as source:
        audio = r.listen(source)
    try:
        return r.recognize_google(audio)
    except sr.UnknownValueError:
        return "(Could not understand audio)"
    except sr.RequestError as e:
        return "Error; {0}".format(e)

In [14]:
games = [["sheep", "ship"], ["eat", "it"], ["she", "see"], ["there", "dare"], ["president", "precedent"]]
random_permutation(games)
count = 0

for i in games[:2]:
    print("\n")
    print("You now pronounce two words:", i[0], "and", i[1])
    for j in range(2):
        if (j == 0):
            print("First, you say:", i[0])
        else:
            print("Second, you say:", i[1])
        recording_now()
        res = detect()
        print("You have said", res)
        if (res == i[j]):
            print("It is CORRECT!")
            count += 1
        else:
            print("Unfornately, it is INCORRECT!")

print("\n")
print("You pronounce correctly", count, "out of", len(games[:2]) * 2)
print("Your score is", count / len(games[:2]) * 100)



You now pronounce two words: she and see
First, you say: she
* recording
* done recording
You have said she
It is CORRECT!
Second, you say: see
* recording
* done recording
You have said see
It is CORRECT!


You now pronounce two words: sheep and ship
First, you say: sheep
* recording
* done recording
You have said shape
Unfornately, it is INCORRECT!
Second, you say: ship
* recording
* done recording
You have said check
Unfornately, it is INCORRECT!


You pronounce correctly 2 out of 4
Your score is 100.0
